# 파트 3: SharePoint 지식 소스 [옵션] - 실제 쉐어포인트 연결이 필요합니다 

파트 1과 2에서는 검색 인덱스를 사용했습니다. 파트 3에서는 `RemoteSharePointKnowledgeSource`를 사용하여 SharePoint 문서에 직접 연결합니다. 이를 통해 먼저 인덱싱하지 않고도 실시간 SharePoint 콘텐츠를 쿼리할 수 있습니다.

## 단계 1: 환경 변수 로드

아래 셀을 실행하여 Azure 리소스의 구성을 로드하고, 생성된 **.venv(3.11.9)** 환경을 선택하세요.

> **⚠️ 문제 해결**
>
> 코드 셀이 멈추고 계속 돌아가면 노트북 상단 툴바에서 **Restart**를 선택하세요. 몇 번 시도해도 문제가 지속되면 VS Code를 완전히 닫고 다시 여세요.

In [1]:
import os

from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

load_dotenv(override=True) # take environment variables from .env.

# Azure AI Search configuration
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"])

# Knowledge base name
knowledge_base_name = "sharepoint-knowledge-base"

# Azure OpenAI configuration
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_KEY"]
azure_openai_chatgpt_deployment = os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT", "gpt-4.1")
azure_openai_chatgpt_model_name = os.getenv("AZURE_OPENAI_CHATGPT_MODEL_NAME", "gpt-4.1")

print("Environment variables loaded")

Environment variables loaded


## 단계 2: 원격 SharePoint 지식 소스 생성

**RemoteSharePointKnowledgeSource**는 먼저 인덱싱하지 않고도 SharePoint 문서에 직접 연결합니다. 이것은 파트 1과 2에서 사용한 `SearchIndexKnowledgeSource`와 다릅니다.

아래 코드는 `RemoteSharePointKnowledgeSourceParameters()`로 SharePoint 지식 소스를 생성합니다. 서비스가 인증과 문서 액세스를 자동으로 처리합니다.

In [2]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import RemoteSharePointKnowledgeSource, RemoteSharePointKnowledgeSourceParameters

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)

ks = RemoteSharePointKnowledgeSource(
    name="sharepoint-knowledge-source",
    description="Knowledge source for SharePoint Documents",
    remote_share_point_parameters=RemoteSharePointKnowledgeSourceParameters()
)
index_client.create_or_update_knowledge_source(knowledge_source=ks)
print(f"Knowledge source '{ks.name}' created or updated successfully.")

Knowledge source 'sharepoint-knowledge-source' created or updated successfully.


## 단계 3: SharePoint 지식 베이스 생성

이 단계에서는 SharePoint 지식 소스를 참조하는 지식 베이스를 생성합니다. 설정은 파트 1과 2와 동일합니다. Azure OpenAI 모델 매개변수를 구성하고, 지식 소스에 대한 참조를 추가하며, AI 기반 답변 생성을 활성화하기 위해 `output_mode=ANSWER_SYNTHESIS`를 설정합니다.

유일한 차이점은 지식 소스 유형입니다. 검색 인덱스를 가리키는 대신, 이 지식 베이스는 SharePoint를 직접 쿼리합니다.

In [3]:
from azure.search.documents.indexes.models import AzureOpenAIVectorizerParameters, KnowledgeBase, KnowledgeBaseAzureOpenAIModel, KnowledgeRetrievalOutputMode, KnowledgeSourceReference

aoai_params = AzureOpenAIVectorizerParameters(
    resource_url=azure_openai_endpoint,
    deployment_name=azure_openai_chatgpt_deployment,
    model_name=azure_openai_chatgpt_model_name,
    api_key=azure_openai_key
)

knowledge_base = KnowledgeBase(
    name=knowledge_base_name,
    models=[KnowledgeBaseAzureOpenAIModel(azure_open_ai_parameters=aoai_params)],
    knowledge_sources=[
        KnowledgeSourceReference(name=ks.name)
    ],
    output_mode=KnowledgeRetrievalOutputMode.ANSWER_SYNTHESIS
)

index_client.create_or_update_knowledge_base(knowledge_base)
print(f"Knowledge base '{knowledge_base_name}' created or updated successfully.")

Knowledge base 'sharepoint-knowledge-base' created or updated successfully.


## 단계 4: SharePoint 문서 쿼리

이제 생성한 지식 베이스를 사용하여 SharePoint 문서를 쿼리할 수 있습니다.

SharePoint를 쿼리할 때 한 가지 중요한 차이점이 있습니다: 지식 베이스가 토큰 소유자를 대신하여 SharePoint에 액세스할 수 있도록 ID 토큰(`x_ms_query_source_authorization`)을 제공해야 합니다. 아래 셀은 Azure 자격 증명을 사용하여 이 토큰을 가져옵니다. 프로덕션 애플리케이션에서 해당 토큰은 앱에 로그인한 사용자로부터 가져옵니다.

두 번째 셀은 테스트 계정과 공유된 SharePoint 문서에서만 답할 수 있는 질문을 실행합니다. 해당 문서를 보려면 [Lab SharePoint](https://lodsprodmca.sharepoint.com/sites/lab511)로 이동하여 Documents 폴더를 여세요.

두 번째 셀을 실행하면 지식 베이스가 질문을 분석하고, 집중된 하위 쿼리로 분해하며, SharePoint를 검색하고, 시맨틱 랭킹을 사용하여 결과를 필터링하고, 인용이 포함된 근거 있는 답변을 합성합니다.

In [4]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

identity_token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://search.azure.com/.default")
token = identity_token_provider()
print("Authentication completed")

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	WorkloadIdentityCredential: WorkloadIdentityCredential authentication unavailable. The workload options are not fully configured. See the troubleshooting guide for more information: https://aka.ms/azsdk/python/identity/workloadidentitycredential/troubleshoot. Missing required arguments: 'tenant_id', 'client_id', 'token_file_path'.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	VisualStudioCodeCredential: VisualStudioCodeCredential requires the 'azure-identity-broker' pa

ClientAuthenticationError: DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	WorkloadIdentityCredential: WorkloadIdentityCredential authentication unavailable. The workload options are not fully configured. See the troubleshooting guide for more information: https://aka.ms/azsdk/python/identity/workloadidentitycredential/troubleshoot. Missing required arguments: 'tenant_id', 'client_id', 'token_file_path'.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	VisualStudioCodeCredential: VisualStudioCodeCredential requires the

In [ ]:
from azure.search.documents.knowledgebases import KnowledgeBaseRetrievalClient
from azure.search.documents.knowledgebases.models import KnowledgeBaseMessage, KnowledgeBaseMessageTextContent, KnowledgeBaseRetrievalRequest, RemoteSharePointKnowledgeSourceParams
from IPython.display import display, Markdown

knowledge_base_client = KnowledgeBaseRetrievalClient(endpoint=endpoint, knowledge_base_name=knowledge_base_name, credential=credential)

sharepoint_ks_params = RemoteSharePointKnowledgeSourceParams(
    knowledge_source_name="sharepoint-knowledge-source",
    include_references=True,
    include_reference_source_data=True
)
req = KnowledgeBaseRetrievalRequest(
    messages=[
        KnowledgeBaseMessage(role="user", content=[KnowledgeBaseMessageTextContent(text="What are major announcements and innovations in Zava Smart Wearables & Clothing Industry in 2025?")])
    ],
    knowledge_source_params=[
        sharepoint_ks_params
    ],
    include_activity=True
)


result = knowledge_base_client.retrieve(retrieval_request=req, x_ms_query_source_authorization=token)
display(Markdown(result.response[0].content[0].text))

## 단계 5: 응답, 참조, 활동 검토

아래 두 셀은 SharePoint 쿼리에서 인용과 활동 로그를 보여줍니다.

참조는 어떤 SharePoint 문서가 사용되었는지 보여줍니다. 각 인용에는 파일 경로와 답변에 기여한 특정 텍스트 스니펫이 포함됩니다.

활동 로그는 내부에서 무슨 일이 일어났는지 보여줍니다: 지식 베이스가 SharePoint에 어떻게 액세스했는지, 어떤 검색을 수행했는지, 결과를 어떻게 순위 지정했는지.

In [ ]:
import json

references = json.dumps([ref.as_dict() for ref in result.references], indent=2)
print(references)

In [ ]:
import pandas as pd

activity_types = [{"type": a.type} for a in result.activity]

df = pd.DataFrame(activity_types)

print("Activity Log Steps")
df

In [ ]:
activity_content = json.dumps([a.as_dict() for a in result.activity], indent=2)
print("Activity Details")
print(activity_content)

## 요약

이 파트에서는 `RemoteSharePointKnowledgeSource`를 사용하여 SharePoint 문서에 직접 연결하는 방법을 배웠습니다. 이를 통해 먼저 인덱싱하지 않고도 실시간 SharePoint 콘텐츠를 쿼리할 수 있습니다.

**기억해야 할 핵심 개념:**
- `RemoteSharePointKnowledgeSource`는 SharePoint 문서를 실시간으로 쿼리합니다
- SharePoint 쿼리에는 ID 토큰(`x_ms_query_source_authorization`)이 필요합니다
- 지식 베이스가 인증과 문서 액세스를 자동으로 처리합니다
- 인용에는 인덱스 참조가 아닌 SharePoint 파일 경로가 포함됩니다

### 다음 단계

➡️ [파트 4: 웹 지식 소스](part4-web-knowledge-source.ipynb)로 계속하여 내부 데이터 소스와 함께 공개 웹 URL을 쿼리하는 방법을 배웁니다.